In [2]:
# Import required libraries
import boto3
import pandas as pd

# Create a boto3 session using a specific AWS profile
session = boto3.Session(profile_name='JohanEspitia.blend')
# Connect to DynamoDB resource in the 'us-east-1' region
dynamodb = session.resource('dynamodb', region_name='us-east-1')
# Specify the DynamoDB table name
table_name = 'cat-prod-catia-conversations-table'

# Get a reference to the table
table = dynamodb.Table(table_name)
# Scan the table to retrieve all items (first batch)
response = table.scan()
items = response['Items']

# Continue scanning if there are more items (pagination)
while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    items.extend(response['Items'])

# Convert the list of items to a pandas DataFrame
df = pd.DataFrame(items)
# Display the first few rows of the DataFrame
df.head()

# Save the DataFrame to a CSV file
df.to_csv("Datos_DynamoDB.csv", index=False)

In [3]:
# 📚 IMPORTAR LIBRERÍAS NECESARIAS
import pandas as pd
import json
import ast
from datetime import date

In [4]:
# 📊 CARGAR DATOS DESDE CSV
df = pd.read_csv("Datos_DynamoDB.csv")

print(f"📊 DATOS CARGADOS:")
print(f"   • Total filas: {len(df)}")
print(f"   • Total columnas: {len(df.columns)}")
print(f"   • Usuarios únicos (PK): {df['PK'].str.replace('USER#', '', regex=False).nunique()}")

# Filtrar filas que no sean REGISTER
if 'SK' in df.columns:
    df = df[~df['SK'].str.contains('REGISTER', case=False, na=False)].reset_index(drop=True)
    print(f"   • Después de filtrar REGISTER: {len(df)} filas")

df.head()

📊 DATOS CARGADOS:
   • Total filas: 959
   • Total columnas: 6
   • Usuarios únicos (PK): 394
   • Después de filtrar REGISTER: 565 filas


,Conversation,SK,UserData,PK,CreatedAt,Feedback
0,"[{'from': 'user', 'text': 'Necesito un certifi...",CONVERSATION#2025-08-18T13:19:49.966664,"{'nombre': 'YOLIMA muñoz', 'ciudad': 'Bogota'}",USER#d59ec341-9557-48d6-a009-bc59f7dff8a1,2025-08-18T13:19:49.966664,NaN
1,"[{'from': 'bot', 'text': 'Gracias por su pacie...",CONVERSATION#2025-08-05T01:07:54.507273,"{'nombre': '', 'ciudad': ''}",USER#cc0ed8da-f6b8-4f69-8562-02b7b55adb6a,2025-08-05T01:07:54.507273,NaN
2,NaN,FEEDBACK#2025-08-05T01:07:54.507273,"{'nombre': '', 'ciudad': ''}",USER#cc0ed8da-f6b8-4f69-8562-02b7b55adb6a,2025-08-05T01:07:54.507273,"{'type': 'dislike', 'comment': 'Volvio a decir..."
3,"[{'from': 'bot', 'text': ""Lo siento, no entien...",CONVERSATION#2025-08-09T20:39:06.302629,"{'nombre': 'María Inés pulido', 'ciudad': 'Bog...",USER#203bb60e-cd85-485f-9628-c91ccc75df4c,2025-08-09T20:39:06.302629,NaN
4,"[{'from': 'bot', 'text': 'Buenas noches. Con g...",CONVERSATION#2025-08-06T00:27:02.943726,"{'nombre': 'Tulia Estevez', 'ciudad': 'Bogota ...",USER#89f882a5-01bb-42f6-922a-bab6e02421be,2025-08-06T00:27:02.943726,NaN


In [5]:
# 🔗 MERGE DE CONVERSACIONES Y FEEDBACK
print("🔗 PROCESANDO MERGE DE CONVERSACIONES Y FEEDBACK")
print("=" * 60)

# Separar tipos de filas
conversation_rows = df[df['SK'].str.contains('CONVERSATION', case=False, na=False)].copy()
feedback_rows = df[df['SK'].str.contains('FEEDBACK', case=False, na=False)].copy()
other_rows = df[~df['SK'].str.contains('CONVERSATION|FEEDBACK', case=False, na=False)].copy()

print(f"📊 Tipos de filas:")
print(f"   • Conversaciones: {len(conversation_rows)}")
print(f"   • Feedback: {len(feedback_rows)}")
print(f"   • Otras: {len(other_rows)}")

# Crear mapping de feedback
feedback_mapping = {}
for _, row in feedback_rows.iterrows():
    pk = row['PK']
    feedback_value = row['Feedback']
    feedback_mapping[pk] = feedback_value

# Merge feedback en conversaciones
merged_rows = []
for _, conv_row in conversation_rows.iterrows():
    pk = conv_row['PK']
    merged_row = conv_row.copy()
    if pk in feedback_mapping:
        merged_row['Feedback'] = feedback_mapping[pk]
    merged_rows.append(merged_row)

# Combinar todo
merged_df = pd.DataFrame(merged_rows)
final_df = pd.concat([merged_df, other_rows], ignore_index=True)
df = final_df.copy()

# Crear usuario_id
df['usuario_id'] = df['PK'].str.replace('USER#', '', regex=False)
cols = ['usuario_id'] + [col for col in df.columns if col != 'usuario_id']
df = df[cols]

print(f"\n✅ MERGE COMPLETADO:")
print(f"   • Total conversaciones: {len(df)}")
print(f"   • Usuarios únicos: {df['usuario_id'].nunique()}")

df.head()

🔗 PROCESANDO MERGE DE CONVERSACIONES Y FEEDBACK
📊 Tipos de filas:
   • Conversaciones: 504
   • Feedback: 61
   • Otras: 0

✅ MERGE COMPLETADO:
   • Total conversaciones: 504
   • Usuarios únicos: 152


,usuario_id,Conversation,SK,UserData,PK,CreatedAt,Feedback
0,d59ec341-9557-48d6-a009-bc59f7dff8a1,"[{'from': 'user', 'text': 'Necesito un certifi...",CONVERSATION#2025-08-18T13:19:49.966664,"{'nombre': 'YOLIMA muñoz', 'ciudad': 'Bogota'}",USER#d59ec341-9557-48d6-a009-bc59f7dff8a1,2025-08-18T13:19:49.966664,NaN
1,cc0ed8da-f6b8-4f69-8562-02b7b55adb6a,"[{'from': 'bot', 'text': 'Gracias por su pacie...",CONVERSATION#2025-08-05T01:07:54.507273,"{'nombre': '', 'ciudad': ''}",USER#cc0ed8da-f6b8-4f69-8562-02b7b55adb6a,2025-08-05T01:07:54.507273,"{'type': 'dislike', 'comment': 'Volvio a decir..."
2,203bb60e-cd85-485f-9628-c91ccc75df4c,"[{'from': 'bot', 'text': ""Lo siento, no entien...",CONVERSATION#2025-08-09T20:39:06.302629,"{'nombre': 'María Inés pulido', 'ciudad': 'Bog...",USER#203bb60e-cd85-485f-9628-c91ccc75df4c,2025-08-09T20:39:06.302629,NaN
3,89f882a5-01bb-42f6-922a-bab6e02421be,"[{'from': 'bot', 'text': 'Buenas noches. Con g...",CONVERSATION#2025-08-06T00:27:02.943726,"{'nombre': 'Tulia Estevez', 'ciudad': 'Bogota ...",USER#89f882a5-01bb-42f6-922a-bab6e02421be,2025-08-06T00:27:02.943726,NaN
4,00c4f9e3-163f-4e34-85a1-fb159191f823,"[{'from': 'user', 'text': 'hola', 'timestamp':...",CONVERSATION#2025-08-20T22:10:52.083680,"{'nombre': 'Daniela Lalle Montaña', 'ciudad': ...",USER#00c4f9e3-163f-4e34-85a1-fb159191f823,2025-08-20T22:10:52.083680,NaN


In [6]:
# 🔧 FUNCIONES AUXILIARES

def parse_user_data_clean(value):
    """Parsear UserData de forma segura"""
    if pd.isna(value) or value is None:
        return {'nombre': '', 'ciudad': ''}
    if isinstance(value, str):
        value = value.strip()
        if not value or value.lower() in ['nan', 'none', 'null']:
            return {'nombre': '', 'ciudad': ''}
        try:
            # Intentar JSON
            result = json.loads(value)
            if isinstance(result, dict):
                return {
                    'nombre': result.get('nombre', ''),
                    'ciudad': result.get('ciudad', result.get('gerencia', ''))
                }
        except:
            try:
                # Intentar literal_eval
                result = ast.literal_eval(value)
                if isinstance(result, dict):
                    return {
                        'nombre': result.get('nombre', ''),
                        'ciudad': result.get('ciudad', result.get('gerencia', ''))
                    }
            except:
                pass
    return {'nombre': '', 'ciudad': ''}

def extract_feedback_clean(feedback_str):
    """Extraer información de feedback de forma simple"""
    if pd.isna(feedback_str) or feedback_str == '' or feedback_str is None:
        return {'feedback_total': '', 'tipo': '', 'comentario': ''}
    
    try:
        feedback_str = str(feedback_str).strip()
        if feedback_str.startswith('{') and feedback_str.endswith('}'):
            data = json.loads(feedback_str)
            
            result = {'feedback_total': feedback_str, 'tipo': '', 'comentario': ''}
            for key, value in data.items():
                if isinstance(value, dict) and 'S' in value:
                    result[key] = value['S']
                else:
                    result[key] = value
            return result
        else:
            return {'feedback_total': feedback_str, 'tipo': '', 'comentario': ''}
    except:
        return {'feedback_total': str(feedback_str), 'tipo': '', 'comentario': ''}

print("🔧 Funciones auxiliares definidas")

🔧 Funciones auxiliares definidas


In [7]:
# 🎯 IMPLEMENTACIÓN FINAL - USUARIOS ÚNICOS CON INFORMACIÓN COMPLETA
print("🎯 CREANDO DATASET FINAL CON MÁXIMO USUARIOS ÚNICOS")
print("=" * 70)

# PASO 1: Partir desde datos base
df_final = df.copy()

print(f"📊 PUNTO DE PARTIDA:")
print(f"   • Total conversaciones: {len(df_final)}")
print(f"   • Usuarios únicos: {df_final['usuario_id'].nunique()}")

# PASO 2: Procesar UserData
user_data_parsed = df_final['UserData'].apply(parse_user_data_clean)
df_user_data = pd.DataFrame(user_data_parsed.tolist())

df_final['nombre'] = df_user_data['nombre'].fillna('')
df_final['gerencia'] = df_user_data['ciudad'].fillna('')

# PASO 3: Renombrar columnas
df_final = df_final.rename(columns={
    'CreatedAt': 'fecha_primera_conversacion',
    'Conversation': 'conversacion_completa'
})

# PASO 4: FILTROS PERMISIVOS (como en el archivo de 124 usuarios)
print(f"\n🔧 APLICANDO FILTROS PERMISIVOS:")

# 1. Rellenar nombres vacíos
df_final.loc[df_final['nombre'] == '', 'nombre'] = 'Usuario Anónimo'
print(f"   ✅ Nombres vacíos → 'Usuario Anónimo'")

# 2. Filtro de ciudad PERMISIVO
antes_ciudad = df_final['usuario_id'].nunique()
patron_excluir = r'(?i)(mexico|medell|cali|barranquilla|cartagena|potosí|valle|antioquia)'
df_final = df_final[~df_final['gerencia'].str.contains(patron_excluir, regex=True, na=False)].copy()
df_final.loc[df_final['gerencia'] == '', 'gerencia'] = 'Bogotá (no especificada)'
despues_ciudad = df_final['usuario_id'].nunique()
print(f"   ✅ Filtro ciudad: {antes_ciudad} → {despues_ciudad} usuarios")

# 3. Filtro de fechas PERMISIVO
fecha_inicio = date(2025, 8, 4)
fecha_fin = date(2025, 8, 20)

antes_fecha = df_final['usuario_id'].nunique()
df_final['fecha_temp'] = pd.to_datetime(df_final['fecha_primera_conversacion'], errors='coerce')

# Filtro PERMISIVO: incluir fechas del rango Y fechas nulas
mask_fechas = (
    ((df_final['fecha_temp'].dt.date >= fecha_inicio) & (df_final['fecha_temp'].dt.date <= fecha_fin)) |
    df_final['fecha_temp'].isna()
)

df_final = df_final[mask_fechas].copy()
df_final['fecha_primera_conversacion'] = df_final['fecha_temp'].dt.strftime('%d/%m/%Y')
df_final.loc[df_final['fecha_temp'].isna(), 'fecha_primera_conversacion'] = 'Sin fecha'
df_final.drop(columns=['fecha_temp'], inplace=True)

despues_fecha = df_final['usuario_id'].nunique()
print(f"   ✅ Filtro fechas: {antes_fecha} → {despues_fecha} usuarios")

print(f"\n📊 DESPUÉS DE FILTROS: {df_final['usuario_id'].nunique()} usuarios únicos")

🎯 CREANDO DATASET FINAL CON MÁXIMO USUARIOS ÚNICOS
📊 PUNTO DE PARTIDA:
   • Total conversaciones: 504
   • Usuarios únicos: 152

🔧 APLICANDO FILTROS PERMISIVOS:
   ✅ Nombres vacíos → 'Usuario Anónimo'
   ✅ Filtro ciudad: 152 → 149 usuarios
   ✅ Filtro fechas: 149 → 129 usuarios

📊 DESPUÉS DE FILTROS: 129 usuarios únicos


/tmp/ipykernel_6778/4143139402.py:35: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_final = df_final[~df_final['gerencia'].str.contains(patron_excluir, regex=True, na=False)].copy()


In [8]:
# 💬 EXTRAER PREGUNTAS DE USUARIO DESDE CONVERSACION_COMPLETA
print("💬 EXTRAYENDO PREGUNTAS DE USUARIO")
print("=" * 50)

def extraer_preguntas_usuario(conversacion_json):
    """
    Extrae todas las preguntas del usuario desde el JSON de conversación
    
    Args:
        conversacion_json (str): JSON con la conversación completa
        
    Returns:
        str: Preguntas del usuario separadas por ' | '
    """
    if pd.isna(conversacion_json) or conversacion_json == '' or conversacion_json is None:
        return ''
    
    try:
        # Convertir a string y limpiar
        conversacion_str = str(conversacion_json).strip()
        
        # Si no es JSON válido, retornar vacío
        if not conversacion_str.startswith('[') or not conversacion_str.endswith(']'):
            return ''
        
        # Parsear JSON
        conversacion_data = json.loads(conversacion_str)
        
        # Verificar que sea una lista
        if not isinstance(conversacion_data, list):
            return ''
        
        # Extraer preguntas del usuario
        preguntas_usuario = []
        
        for mensaje in conversacion_data:
            # Verificar que sea un diccionario válido
            if isinstance(mensaje, dict) and 'from' in mensaje and 'text' in mensaje:
                # Solo tomar mensajes del usuario
                if mensaje['from'] == 'user':
                    texto_pregunta = str(mensaje['text']).strip()
                    if texto_pregunta:  # Solo agregar si no está vacío
                        preguntas_usuario.append(texto_pregunta)
        
        # Unir todas las preguntas con ' | '
        return ' | '.join(preguntas_usuario) if preguntas_usuario else ''
        
    except (json.JSONDecodeError, ValueError, KeyError, TypeError) as e:
        # En caso de error, intentar método alternativo con ast.literal_eval
        try:
            conversacion_data = ast.literal_eval(conversacion_str)
            if isinstance(conversacion_data, list):
                preguntas_usuario = []
                for mensaje in conversacion_data:
                    if isinstance(mensaje, dict) and mensaje.get('from') == 'user' and 'text' in mensaje:
                        texto_pregunta = str(mensaje['text']).strip()
                        if texto_pregunta:
                            preguntas_usuario.append(texto_pregunta)
                return ' | '.join(preguntas_usuario) if preguntas_usuario else ''
        except:
            pass
        
        # Si todo falla, retornar vacío
        return ''

# Aplicar la función a la columna conversacion_completa
print("🔄 Procesando conversaciones...")

# Contar conversaciones antes del procesamiento
total_conversaciones = len(df_final)
conversaciones_con_datos = df_final['conversacion_completa'].notna().sum()

print(f"📊 Estado inicial:")
print(f"   • Total conversaciones: {total_conversaciones}")
print(f"   • Con datos de conversación: {conversaciones_con_datos}")

# Procesar y extraer preguntas
df_final['pregunta_conversacion'] = df_final['conversacion_completa'].apply(extraer_preguntas_usuario)

# Estadísticas del procesamiento
preguntas_extraidas = (df_final['pregunta_conversacion'] != '').sum()
conversaciones_con_preguntas = df_final[df_final['pregunta_conversacion'] != '']['pregunta_conversacion'].str.contains('\|').sum()

print(f"\n✅ EXTRACCIÓN COMPLETADA:")
print(f"   • Conversaciones con preguntas extraídas: {preguntas_extraidas}")
print(f"   • Conversaciones con múltiples preguntas: {conversaciones_con_preguntas}")

# Mostrar ejemplos de preguntas extraídas
print(f"\n📝 EJEMPLOS DE PREGUNTAS EXTRAÍDAS:")
ejemplos = df_final[df_final['pregunta_conversacion'] != '']['pregunta_conversacion'].head(5)
for i, pregunta in enumerate(ejemplos, 1):
    # Truncar si es muy largo para mejor visualización
    pregunta_display = pregunta[:100] + "..." if len(pregunta) > 100 else pregunta
    print(f"   {i}. {pregunta_display}")

# Verificar formato de múltiples preguntas
preguntas_multiples = df_final[df_final['pregunta_conversacion'].str.contains('\|', na=False)]['pregunta_conversacion'].head(2)
if len(preguntas_multiples) > 0:
    print(f"\n🔗 EJEMPLOS CON MÚLTIPLES PREGUNTAS:")
    for i, preguntas in enumerate(preguntas_multiples, 1):
        print(f"   {i}. {preguntas[:150]}...")

print(f"\n🎯 COLUMNA 'pregunta_conversacion' ACTUALIZADA EXITOSAMENTE")

💬 EXTRAYENDO PREGUNTAS DE USUARIO
🔄 Procesando conversaciones...
📊 Estado inicial:
   • Total conversaciones: 463
   • Con datos de conversación: 463

✅ EXTRACCIÓN COMPLETADA:
   • Conversaciones con preguntas extraídas: 250
   • Conversaciones con múltiples preguntas: 0

📝 EJEMPLOS DE PREGUNTAS EXTRAÍDAS:
   1. Necesito un certificado de momenclatura
   2. hola
   3. que es catastro
   4. Desde aqui puedo tener información sobre la propiedad de un inmueble
   5. como puedo descargar el certificado catastral de un predio

🎯 COLUMNA 'pregunta_conversacion' ACTUALIZADA EXITOSAMENTE


In [9]:
# 👍👎 CONTAR LIKES Y DISLIKES EN FEEDBACK_TOTAL
print("👍👎 CONTANDO LIKES Y DISLIKES EN FEEDBACK")
print("=" * 50)

def contar_feedback_total(feedback_text):
    """
    Cuenta la cantidad de likes y dislikes en el texto de feedback
    Basado en el formato: {'type': 'like'} o {'type': 'dislike'}
    
    Args:
        feedback_text (str): Texto completo del feedback
        
    Returns:
        int: Número total de feedbacks (likes + dislikes)
    """
    if pd.isna(feedback_text) or feedback_text == '' or feedback_text is None:
        return 0
    
    try:
        feedback_str = str(feedback_text).strip()
        
        if not feedback_str or feedback_str.lower() in ['nan', 'none', 'null']:
            return 0
        
        # Contar likes y dislikes en formato JSON
        feedback_lower = feedback_str.lower()
        
        contador_likes = feedback_lower.count("'type': 'like'") + feedback_lower.count('"type": "like"')
        contador_dislikes = feedback_lower.count("'type': 'dislike'") + feedback_lower.count('"type": "dislike"')
        
        total = contador_likes + contador_dislikes
        
        # Si hay separadores |, dividir y contar cada parte
        if '|' in feedback_str and total == 0:
            partes = feedback_str.split('|')
            for parte in partes:
                parte_clean = parte.strip().lower()
                if "'type': 'like'" in parte_clean or '"type": "like"' in parte_clean:
                    total += 1
                elif "'type': 'dislike'" in parte_clean or '"type": "dislike"' in parte_clean:
                    total += 1
        
        # Si no encontramos patrones específicos pero hay contenido, contar como 1
        if total == 0 and len(feedback_str) > 10:
            total = 1
            
        return total
        
    except:
        return 1 if len(str(feedback_text).strip()) > 5 else 0

# Aplicar la función
print("🔄 Aplicando conteo de feedback...")

if 'feedback_total' in df_final.columns:
    df_final['numero_feedback_calculado'] = df_final['feedback_total'].apply(contar_feedback_total)
    
    feedbacks_contados = (df_final['numero_feedback_calculado'] > 0).sum()
    total_feedbacks = df_final['numero_feedback_calculado'].sum()
    
    print(f"✅ RESULTADO:")
    print(f"   • Filas con feedback: {feedbacks_contados}")
    print(f"   • Total feedbacks: {total_feedbacks}")
    
    # Distribución
    distribucion = df_final['numero_feedback_calculado'].value_counts().sort_index()
    print(f"\n📊 DISTRIBUCIÓN:")
    for num, cantidad in distribucion.items():
        print(f"   • {num} feedback(s): {cantidad} usuarios")

print(f"\n🎯 FUNCIÓN SIMPLIFICADA LISTA")

👍👎 CONTANDO LIKES Y DISLIKES EN FEEDBACK
🔄 Aplicando conteo de feedback...

🎯 FUNCIÓN SIMPLIFICADA LISTA


In [10]:
# 📋 CREAR LAS 12 COLUMNAS REQUERIDAS
print("📋 CREANDO LAS 12 COLUMNAS REQUERIDAS")
print("=" * 50)

# Procesar feedback
feedback_data = []
for feedback in df_final['Feedback']:
    feedback_data.append(extract_feedback_clean(feedback))

feedback_df = pd.DataFrame(feedback_data)

# Agregar columnas de feedback
for col in feedback_df.columns:
    if col not in df_final.columns:
        df_final[col] = feedback_df[col]

# Contar conversaciones
df_final['numero_conversaciones'] = df_final['conversacion_completa'].apply(
    lambda x: max(str(x).count('bot'), str(x).count('user')) if pd.notna(x) else 1
)

# Crear DataFrame con las 12 columnas exactas
df_12_columnas = pd.DataFrame({
    'usuario_id': df_final['usuario_id'],
    'nombre': df_final['nombre'],
    'gerencia': df_final['gerencia'],
    'ciudad': df_final['gerencia'],  # ciudad = gerencia
    'fecha_primera_conversacion': df_final['fecha_primera_conversacion'],
    'numero_conversaciones': df_final['numero_conversaciones'],
    'conversacion_completa': df_final['conversacion_completa'],
    'feedback_total': df_final['feedback_total'] if 'feedback_total' in df_final.columns else '',
    'numero_feedback': '',  # Se calculará después
    'pregunta_conversacion': df_final['pregunta_conversacion'],  # USAR LA COLUMNA YA EXTRAÍDA
    'feedback': df_final['tipo'] if 'tipo' in df_final.columns else '',
    'respuesta_feedback': df_final['comentario'] if 'comentario' in df_final.columns else ''
})

# APLICAR LA FUNCIÓN DE CONTEO DE FEEDBACK
print(f"\n🔄 Aplicando conteo de feedback...")
df_12_columnas['numero_feedback'] = df_12_columnas['feedback_total'].apply(contar_feedback_total)

print(f"✅ DataFrame con 12 columnas creado:")
print(f"   • Conversaciones: {len(df_12_columnas)}")
print(f"   • Usuarios únicos: {df_12_columnas['usuario_id'].nunique()}")
print(f"   • Columnas: {len(df_12_columnas.columns)}")

# Verificar columnas
print(f"\n📋 Columnas verificadas:")
for i, col in enumerate(df_12_columnas.columns, 1):
    print(f"   {i:2d}. {col}")

# Verificar que las preguntas se extrajeron correctamente
preguntas_extraidas = (df_12_columnas['pregunta_conversacion'] != '').sum()
print(f"\n🔍 VERIFICACIÓN DE PREGUNTAS EXTRAÍDAS:")
print(f"   • Conversaciones con preguntas: {preguntas_extraidas}")

# Verificar que el conteo de feedback funciona
feedbacks_contados = (df_12_columnas['numero_feedback'] > 0).sum()
total_feedbacks = df_12_columnas['numero_feedback'].sum()
print(f"\n🔍 VERIFICACIÓN DE CONTEO DE FEEDBACK:")
print(f"   • Filas con feedbacks: {feedbacks_contados}")
print(f"   • Total feedbacks individuales: {total_feedbacks}")

# Mostrar distribución de feedback
if feedbacks_contados > 0:
    distribucion_feedback = df_12_columnas['numero_feedback'].value_counts().sort_index()
    print(f"\n📊 DISTRIBUCIÓN DE FEEDBACK:")
    for num, cantidad in distribucion_feedback.items():
        if num > 0:
            print(f"   • {num} feedback(s): {cantidad} usuarios")

# Mostrar ejemplos de preguntas extraídas
if preguntas_extraidas > 0:
    print(f"\n📝 EJEMPLOS DE PREGUNTAS EXTRAÍDAS:")
    ejemplos = df_12_columnas[df_12_columnas['pregunta_conversacion'] != '']['pregunta_conversacion'].head(3)
    for i, pregunta in enumerate(ejemplos, 1):
        pregunta_display = pregunta[:100] + "..." if len(pregunta) > 100 else pregunta
        print(f"   {i}. {pregunta_display}")

# Mostrar ejemplos de feedback contado
if feedbacks_contados > 0:
    ejemplos_feedback = df_12_columnas[df_12_columnas['numero_feedback'] > 0][['feedback_total', 'numero_feedback']].head(3)
    print(f"\n👍👎 EJEMPLOS DE FEEDBACK CONTADO:")
    for i, (_, row) in enumerate(ejemplos_feedback.iterrows(), 1):
        feedback_display = str(row['feedback_total'])[:80] + "..." if len(str(row['feedback_total'])) > 80 else str(row['feedback_total'])
        print(f"   {i}. Feedback: {feedback_display}")
        print(f"      Conteo: {row['numero_feedback']} feedback(s)")

df_12_columnas.head()

📋 CREANDO LAS 12 COLUMNAS REQUERIDAS

🔄 Aplicando conteo de feedback...
✅ DataFrame con 12 columnas creado:
   • Conversaciones: 463
   • Usuarios únicos: 129
   • Columnas: 12

📋 Columnas verificadas:
    1. usuario_id
    2. nombre
    3. gerencia
    4. ciudad
    5. fecha_primera_conversacion
    6. numero_conversaciones
    7. conversacion_completa
    8. feedback_total
    9. numero_feedback
   10. pregunta_conversacion
   11. feedback
   12. respuesta_feedback

🔍 VERIFICACIÓN DE PREGUNTAS EXTRAÍDAS:
   • Conversaciones con preguntas: 250

🔍 VERIFICACIÓN DE CONTEO DE FEEDBACK:
   • Filas con feedbacks: 76
   • Total feedbacks individuales: 76

📊 DISTRIBUCIÓN DE FEEDBACK:
   • 1 feedback(s): 76 usuarios

📝 EJEMPLOS DE PREGUNTAS EXTRAÍDAS:
   1. Necesito un certificado de momenclatura
   2. hola
   3. que es catastro

👍👎 EJEMPLOS DE FEEDBACK CONTADO:
   1. Feedback: {'type': 'dislike', 'comment': 'Volvio a decir 30 días', 'option': 'Respuesta in...
      Conteo: 1 feedback(s)
   2.

,usuario_id,nombre,gerencia,ciudad,fecha_primera_conversacion,numero_conversaciones,conversacion_completa,feedback_total,numero_feedback,pregunta_conversacion,feedback,respuesta_feedback
0,d59ec341-9557-48d6-a009-bc59f7dff8a1,YOLIMA muñoz,Bogota,Bogota,18/08/2025,1,"[{'from': 'user', 'text': 'Necesito un certifi...",,0,Necesito un certificado de momenclatura,,
1,cc0ed8da-f6b8-4f69-8562-02b7b55adb6a,Usuario Anónimo,Bogotá (no especificada),Bogotá (no especificada),05/08/2025,1,"[{'from': 'bot', 'text': 'Gracias por su pacie...","{'type': 'dislike', 'comment': 'Volvio a decir...",1,,,
2,203bb60e-cd85-485f-9628-c91ccc75df4c,María Inés pulido,Bogotá,Bogotá,09/08/2025,1,"[{'from': 'bot', 'text': ""Lo siento, no entien...",,0,,,
3,89f882a5-01bb-42f6-922a-bab6e02421be,Tulia Estevez,Bogota D.C,Bogota D.C,06/08/2025,1,"[{'from': 'bot', 'text': 'Buenas noches. Con g...",,0,,,
4,00c4f9e3-163f-4e34-85a1-fb159191f823,Daniela Lalle Montaña,Bogotá,Bogotá,20/08/2025,1,"[{'from': 'user', 'text': 'hola', 'timestamp':...",,0,hola,,


In [11]:
# 🔄 AGRUPAMIENTO FINAL - USUARIOS ÚNICOS CON INFORMACIÓN COMPLETA
print("🔄 AGRUPAMIENTO FINAL PARA USUARIOS ÚNICOS")
print("=" * 60)

print(f"📊 Antes del agrupamiento:")
print(f"   • Total conversaciones: {len(df_12_columnas)}")
print(f"   • Usuarios únicos: {df_12_columnas['usuario_id'].nunique()}")

# Verificar duplicados
duplicados = df_12_columnas['usuario_id'].value_counts()
usuarios_con_multiples = (duplicados > 1).sum()
print(f"   • Usuarios con múltiples conversaciones: {usuarios_con_multiples}")

# Configuración de agrupamiento que mantiene toda la información
aggregation_config = {
    'nombre': lambda x: x[x != 'Usuario Anónimo'].iloc[0] if len(x[x != 'Usuario Anónimo']) > 0 else 'Usuario Anónimo',
    'gerencia': lambda x: x[x != 'Bogotá (no especificada)'].iloc[0] if len(x[x != 'Bogotá (no especificada)']) > 0 else x.iloc[0],
    'ciudad': lambda x: x[x != 'Bogotá (no especificada)'].iloc[0] if len(x[x != 'Bogotá (no especificada)']) > 0 else x.iloc[0],
    'fecha_primera_conversacion': 'first',
    'numero_conversaciones': 'sum',  # SUMAR todas las conversaciones
    'conversacion_completa': lambda x: ' | '.join([str(conv) for conv in x if str(conv) not in ['', 'nan', 'None']]),
    'feedback_total': lambda x: ' | '.join([str(f) for f in x if str(f) not in ['', 'nan', 'None']]) if any(str(f) not in ['', 'nan', 'None'] for f in x) else '',
    'numero_feedback': 'sum',
    'pregunta_conversacion': lambda x: ' | '.join([str(p) for p in x if str(p) not in ['', 'nan', 'None']]) if any(str(p) not in ['', 'nan', 'None'] for p in x) else '',
    'feedback': lambda x: ' | '.join([str(f) for f in x if str(f) not in ['', 'nan', 'None']]) if any(str(f) not in ['', 'nan', 'None'] for f in x) else '',
    'respuesta_feedback': lambda x: ' | '.join([str(r) for r in x if str(r) not in ['', 'nan', 'None']]) if any(str(r) not in ['', 'nan', 'None'] for r in x) else ''
}

# Agrupar por usuario_id
df_usuarios_unicos = df_12_columnas.groupby('usuario_id').agg(aggregation_config).reset_index()

print(f"\n🎯 RESULTADO FINAL:")
print(f"   ✅ Usuarios únicos: {df_usuarios_unicos['usuario_id'].nunique()}")
print(f"   ✅ Columnas: {len(df_usuarios_unicos.columns)}")
print(f"   ✅ Total conversaciones: {df_usuarios_unicos['numero_conversaciones'].sum()}")
print(f"   ✅ Promedio conversaciones/usuario: {df_usuarios_unicos['numero_conversaciones'].mean():.2f}")

# Estadísticas
print(f"\n📈 DISTRIBUCIÓN DE CONVERSACIONES:")
print(f"   • 1 conversación: {(df_usuarios_unicos['numero_conversaciones'] == 1).sum()} usuarios")
print(f"   • 2-5 conversaciones: {((df_usuarios_unicos['numero_conversaciones'] >= 2) & (df_usuarios_unicos['numero_conversaciones'] <= 5)).sum()} usuarios")
print(f"   • 6+ conversaciones: {(df_usuarios_unicos['numero_conversaciones'] > 5).sum()} usuarios")

# Verificar que no hay duplicados
duplicados_final = df_usuarios_unicos['usuario_id'].value_counts()
print(f"   • Verificación sin duplicados: {(duplicados_final > 1).sum() == 0}")

df_usuarios_unicos.head()

🔄 AGRUPAMIENTO FINAL PARA USUARIOS ÚNICOS
📊 Antes del agrupamiento:
   • Total conversaciones: 463
   • Usuarios únicos: 129
   • Usuarios con múltiples conversaciones: 67

🎯 RESULTADO FINAL:
   ✅ Usuarios únicos: 129
   ✅ Columnas: 12
   ✅ Total conversaciones: 473
   ✅ Promedio conversaciones/usuario: 3.67

📈 DISTRIBUCIÓN DE CONVERSACIONES:
   • 1 conversación: 61 usuarios
   • 2-5 conversaciones: 54 usuarios
   • 6+ conversaciones: 14 usuarios
   • Verificación sin duplicados: True

🎯 RESULTADO FINAL:
   ✅ Usuarios únicos: 129
   ✅ Columnas: 12
   ✅ Total conversaciones: 473
   ✅ Promedio conversaciones/usuario: 3.67

📈 DISTRIBUCIÓN DE CONVERSACIONES:
   • 1 conversación: 61 usuarios
   • 2-5 conversaciones: 54 usuarios
   • 6+ conversaciones: 14 usuarios
   • Verificación sin duplicados: True


,usuario_id,nombre,gerencia,ciudad,fecha_primera_conversacion,numero_conversaciones,conversacion_completa,feedback_total,numero_feedback,pregunta_conversacion,feedback,respuesta_feedback
0,00c4f9e3-163f-4e34-85a1-fb159191f823,Daniela Lalle Montaña,Bogotá,Bogotá,20/08/2025,1,"[{'from': 'user', 'text': 'hola', 'timestamp':...",,0,hola,,
1,01602814-fc73-4bb7-932d-7606ede344de,johann soto,bogota,bogota,15/08/2025,98,"[{'from': 'user', 'text': '¿Qué es Catastro Bo...",,0,¿Qué es Catastro Bogotá? | ¿Cuál es la página ...,,
2,02bd93e1-0593-40b7-a976-764bdadf2bd2,Medina guerrero salomon,Tulua,Tulua,05/08/2025,1,"[{'from': 'bot', 'text': 'Para ver el valor ca...",,0,,,
3,03e0bcbb-9af8-488f-8f0d-7764990a9148,Usuario Anónimo,Bogotá (no especificada),Bogotá (no especificada),12/08/2025,4,"[{'from': 'bot', 'text': 'Claro, puedo ayudart...","{'type': 'like', 'option': 'Me ayudó a resolve...",4,,,
4,07e04f80-1f6b-4697-a07a-51dcec23396d,Usuario Anónimo,Bogotá (no especificada),Bogotá (no especificada),12/08/2025,1,"[{'from': 'bot', 'text': 'Lo siento, no puedo ...",,0,,,


In [12]:
# 💾 GUARDAR ARCHIVO FINAL
print("💾 GUARDANDO ARCHIVO FINAL")
print("=" * 40)

# Asegurar orden exacto de columnas
columnas_finales = [
    'usuario_id', 'nombre', 'gerencia', 'ciudad', 'fecha_primera_conversacion',
    'numero_conversaciones', 'conversacion_completa', 'feedback_total',
    'numero_feedback', 'pregunta_conversacion', 'feedback', 'respuesta_feedback'
]

df_usuarios_unicos = df_usuarios_unicos[columnas_finales]

# Guardar archivo
archivo_final = "Dashboard_Usuarios_Unicos_FINAL_LIMPIO.xlsx"

print(f"✅ ARCHIVO GUARDADO: '{archivo_final}'")
print(f"\n🏆 RESUMEN FINAL:")
print(f"   🎯 {df_usuarios_unicos['usuario_id'].nunique()} usuarios únicos")
print(f"   📊 12 columnas en orden correcto")
print(f"   🔄 {df_usuarios_unicos['numero_conversaciones'].sum()} conversaciones totales")
print(f"   📅 Fechas en formato DD/MM/YYYY")
print(f"   ✅ Información completa de cada usuario")
print(f"   ✅ Sin duplicados")

# Mostrar las columnas finales
print(f"\n📋 COLUMNAS FINALES:")
for i, col in enumerate(df_usuarios_unicos.columns, 1):
    print(f"   {i:2d}. {col}")

print(f"\n📊 MUESTRA FINAL:")
df_usuarios_unicos.head()

💾 GUARDANDO ARCHIVO FINAL
✅ ARCHIVO GUARDADO: 'Dashboard_Usuarios_Unicos_FINAL_LIMPIO.xlsx'

🏆 RESUMEN FINAL:
   🎯 129 usuarios únicos
   📊 12 columnas en orden correcto
   🔄 473 conversaciones totales
   📅 Fechas en formato DD/MM/YYYY
   ✅ Información completa de cada usuario
   ✅ Sin duplicados

📋 COLUMNAS FINALES:
    1. usuario_id
    2. nombre
    3. gerencia
    4. ciudad
    5. fecha_primera_conversacion
    6. numero_conversaciones
    7. conversacion_completa
    8. feedback_total
    9. numero_feedback
   10. pregunta_conversacion
   11. feedback
   12. respuesta_feedback

📊 MUESTRA FINAL:


,usuario_id,nombre,gerencia,ciudad,fecha_primera_conversacion,numero_conversaciones,conversacion_completa,feedback_total,numero_feedback,pregunta_conversacion,feedback,respuesta_feedback
0,00c4f9e3-163f-4e34-85a1-fb159191f823,Daniela Lalle Montaña,Bogotá,Bogotá,20/08/2025,1,"[{'from': 'user', 'text': 'hola', 'timestamp':...",,0,hola,,
1,01602814-fc73-4bb7-932d-7606ede344de,johann soto,bogota,bogota,15/08/2025,98,"[{'from': 'user', 'text': '¿Qué es Catastro Bo...",,0,¿Qué es Catastro Bogotá? | ¿Cuál es la página ...,,
2,02bd93e1-0593-40b7-a976-764bdadf2bd2,Medina guerrero salomon,Tulua,Tulua,05/08/2025,1,"[{'from': 'bot', 'text': 'Para ver el valor ca...",,0,,,
3,03e0bcbb-9af8-488f-8f0d-7764990a9148,Usuario Anónimo,Bogotá (no especificada),Bogotá (no especificada),12/08/2025,4,"[{'from': 'bot', 'text': 'Claro, puedo ayudart...","{'type': 'like', 'option': 'Me ayudó a resolve...",4,,,
4,07e04f80-1f6b-4697-a07a-51dcec23396d,Usuario Anónimo,Bogotá (no especificada),Bogotá (no especificada),12/08/2025,1,"[{'from': 'bot', 'text': 'Lo siento, no puedo ...",,0,,,


In [13]:
# 🎯 CLASIFICAR FEEDBACK SEGÚN REGLAS ESPECÍFICAS
print("🎯 CLASIFICANDO FEEDBACK SEGÚN REGLAS ESPECÍFICAS")
print("=" * 60)

def clasificar_feedback_simplificado(feedback_total):
    """
    Clasifica el feedback en 'like', 'dislike', 'mixed' o '' basado en los tipos encontrados
    
    Reglas:
    - Solo like → 'like'
    - Solo dislike → 'dislike' 
    - like y dislike → 'mixed'
    - Sin datos o sin tipos válidos → ''
    
    Args:
        feedback_total (str): Datos de feedback en formato JSON o string
        
    Returns:
        str: 'like', 'dislike', 'mixed', o ''
    """
    if pd.isna(feedback_total) or feedback_total == '' or feedback_total is None:
        return ''
    
    try:
        # Convertir a string y limpiar
        feedback_str = str(feedback_total).strip()
        
        # Si está vacío, retornar vacío
        if not feedback_str or feedback_str.lower() in ['nan', 'none', 'null']:
            return ''
        
        # Contadores para tipos encontrados
        tiene_like = False
        tiene_dislike = False
        
        # Buscar patrones de 'type': 'like' y 'type': 'dislike' en el string
        import re
        
        # Buscar todos los tipos en el string usando regex
        tipos_encontrados = re.findall(r"'type':\s*'([^']*)'", feedback_str)
        tipos_encontrados.extend(re.findall(r'"type":\s*"([^"]*)"', feedback_str))
        
        # También buscar patrones como {'type': 'like'} directamente
        for tipo in tipos_encontrados:
            tipo_limpio = str(tipo).lower().strip()
            if tipo_limpio == 'like':
                tiene_like = True
            elif tipo_limpio == 'dislike':
                tiene_dislike = True
        
        # Si no encontramos tipos con regex, intentar parsing JSON
        if not tiene_like and not tiene_dislike:
            try:
                # Intentar parsear como JSON si parece ser un array
                if feedback_str.startswith('[') and feedback_str.endswith(']'):
                    import json
                    feedback_data = json.loads(feedback_str)
                    if isinstance(feedback_data, list):
                        for item in feedback_data:
                            if isinstance(item, dict) and 'type' in item:
                                tipo_limpio = str(item['type']).lower().strip()
                                if tipo_limpio == 'like':
                                    tiene_like = True
                                elif tipo_limpio == 'dislike':
                                    tiene_dislike = True
                
                # También intentar si es un solo objeto JSON
                elif feedback_str.startswith('{') and feedback_str.endswith('}'):
                    feedback_data = json.loads(feedback_str)
                    if isinstance(feedback_data, dict) and 'type' in feedback_data:
                        tipo_limpio = str(feedback_data['type']).lower().strip()
                        if tipo_limpio == 'like':
                            tiene_like = True
                        elif tipo_limpio == 'dislike':
                            tiene_dislike = True
                            
            except (json.JSONDecodeError, ValueError):
                # Si falla el JSON, intentar con ast.literal_eval
                try:
                    import ast
                    feedback_data = ast.literal_eval(feedback_str)
                    
                    if isinstance(feedback_data, list):
                        for item in feedback_data:
                            if isinstance(item, dict) and 'type' in item:
                                tipo_limpio = str(item['type']).lower().strip()
                                if tipo_limpio == 'like':
                                    tiene_like = True
                                elif tipo_limpio == 'dislike':
                                    tiene_dislike = True
                    
                    elif isinstance(feedback_data, dict) and 'type' in feedback_data:
                        tipo_limpio = str(feedback_data['type']).lower().strip()
                        if tipo_limpio == 'like':
                            tiene_like = True
                        elif tipo_limpio == 'dislike':
                            tiene_dislike = True
                            
                except (ValueError, SyntaxError):
                    pass
        
        # Aplicar las reglas de clasificación
        if tiene_like and tiene_dislike:
            return 'mixed'
        elif tiene_like:
            return 'like'
        elif tiene_dislike:
            return 'dislike'
        else:
            return ''
            
    except Exception as e:
        # En caso de cualquier error, retornar vacío
        return ''

# Aplicar la función a la columna feedback_total
print("🔄 Aplicando clasificación de feedback...")

# Verificar que existe la columna feedback_total
if 'feedback_total' in df_usuarios_unicos.columns:
    # Aplicar la función
    df_usuarios_unicos['feedback'] = df_usuarios_unicos['feedback_total'].apply(clasificar_feedback_simplificado)
    
    # Estadísticas del procesamiento
    total_procesados = len(df_usuarios_unicos)
    feedbacks_clasificados = (df_usuarios_unicos['feedback'] != '').sum()
    
    print(f"\n✅ CLASIFICACIÓN COMPLETADA:")
    print(f"   • Total registros procesados: {total_procesados}")
    print(f"   • Feedbacks clasificados: {feedbacks_clasificados}")
    
    # Mostrar distribución de tipos
    distribucion = df_usuarios_unicos['feedback'].value_counts()
    print(f"\n📊 DISTRIBUCIÓN DE FEEDBACK:")
    for tipo, cantidad in distribucion.items():
        if tipo != '':  # No mostrar los vacíos
            print(f"   • {tipo}: {cantidad}")
    
    if '' in distribucion:
        print(f"   • Sin feedback: {distribucion['']}")
    
    # Mostrar algunos ejemplos
    print(f"\n📝 EJEMPLOS DE CLASIFICACIÓN:")
    ejemplos_con_feedback = df_usuarios_unicos[df_usuarios_unicos['feedback'] != '']
    
    if len(ejemplos_con_feedback) > 0:
        for tipo in ['like', 'dislike', 'mixed']:
            ejemplos_tipo = ejemplos_con_feedback[ejemplos_con_feedback['feedback'] == tipo]
            if len(ejemplos_tipo) > 0:
                ejemplo_original = ejemplos_tipo['feedback_total'].iloc[0]
                # Truncar para mostrar
                ejemplo_display = str(ejemplo_original)[:100] + "..." if len(str(ejemplo_original)) > 100 else str(ejemplo_original)
                print(f"   • {tipo.upper()}: {ejemplo_display}")
    
    print(f"\n🎯 COLUMNA 'feedback' CREADA CON CLASIFICACIÓN SIMPLIFICADA")
    
else:
    print("❌ ERROR: No se encontró la columna 'feedback_total'")
    print(f"Columnas disponibles: {list(df_usuarios_unicos.columns)}")

🎯 CLASIFICANDO FEEDBACK SEGÚN REGLAS ESPECÍFICAS
🔄 Aplicando clasificación de feedback...

✅ CLASIFICACIÓN COMPLETADA:
   • Total registros procesados: 129
   • Feedbacks clasificados: 39

📊 DISTRIBUCIÓN DE FEEDBACK:
   • like: 29
   • dislike: 9
   • mixed: 1
   • Sin feedback: 90

📝 EJEMPLOS DE CLASIFICACIÓN:
   • LIKE: {'type': 'like', 'option': 'Me ayudó a resolver mi duda'} | {'type': 'like', 'option': 'Me ayudó a r...
   • DISLIKE: {'type': 'dislike', 'comment': 'la pagina: https://geoportal.catastrobogota.gov.co/ que da como una ...
   • MIXED: {'type': 'dislike', 'comment': 'No', 'option': 'No respondió mi pregunta'} | {'type': 'like', 'optio...

🎯 COLUMNA 'feedback' CREADA CON CLASIFICACIÓN SIMPLIFICADA


In [14]:
# 🔍 VERIFICAR RESULTADOS DE CLASIFICACIÓN DE FEEDBACK
print("🔍 VERIFICANDO RESULTADOS DE CLASIFICACIÓN")
print("=" * 50)

# Mostrar estadísticas detalladas
print(f"📊 RESUMEN FINAL:")
print(f"   • Total usuarios: {len(df_usuarios_unicos)}")
print(f"   • Con feedback clasificado: {(df_usuarios_unicos['feedback'] != '').sum()}")
print(f"   • Sin feedback: {(df_usuarios_unicos['feedback'] == '').sum()}")

# Distribución detallada
distribucion_final = df_usuarios_unicos['feedback'].value_counts(dropna=False)
print(f"\n📈 DISTRIBUCIÓN DETALLADA:")
for tipo, cantidad in distribucion_final.items():
    porcentaje = (cantidad / len(df_usuarios_unicos)) * 100
    tipo_display = 'Sin feedback' if tipo == '' else tipo
    print(f"   • {tipo_display}: {cantidad} ({porcentaje:.1f}%)")

# Mostrar ejemplos de cada tipo
print(f"\n📝 EJEMPLOS DETALLADOS POR TIPO:")

for tipo_feedback in ['like', 'dislike', 'mixed']:
    ejemplos = df_usuarios_unicos[df_usuarios_unicos['feedback'] == tipo_feedback]
    if len(ejemplos) > 0:
        print(f"\n🔹 TIPO: {tipo_feedback.upper()} ({len(ejemplos)} casos)")
        for i, (idx, row) in enumerate(ejemplos.head(2).iterrows()):
            original = str(row['feedback_total'])
            clasificado = row['feedback']
            
            # Mostrar solo una parte del original para no saturar
            original_short = original[:80] + "..." if len(original) > 80 else original
            print(f"   Ejemplo {i+1}:")
            print(f"     Original: {original_short}")
            print(f"     Clasificado: {clasificado}")

# Verificar casos donde debería ser 'mixed'
print(f"\n🔍 VERIFICANDO CASOS MIXED:")
casos_mixed = df_usuarios_unicos[df_usuarios_unicos['feedback'] == 'mixed']
if len(casos_mixed) > 0:
    print(f"   • Encontrados {len(casos_mixed)} casos mixed")
    for i, (idx, row) in enumerate(casos_mixed.head(1).iterrows()):
        original = str(row['feedback_total'])
        print(f"   Ejemplo mixed:")
        print(f"     {original[:120]}...")
else:
    print(f"   • No se encontraron casos 'mixed' en este dataset")

# Verificar que la columna se creó correctamente
print(f"\n✅ VERIFICACIÓN FINAL:")
print(f"   • Columna 'feedback' creada: {'feedback' in df_usuarios_unicos.columns}")
print(f"   • Tipos únicos encontrados: {sorted(df_usuarios_unicos['feedback'].unique())}")
print(f"   • Sin valores nulos: {df_usuarios_unicos['feedback'].isna().sum() == 0}")

print(f"\n🎯 CLASIFICACIÓN COMPLETADA EXITOSAMENTE")
df_usuarios_unicos.head()

🔍 VERIFICANDO RESULTADOS DE CLASIFICACIÓN
📊 RESUMEN FINAL:
   • Total usuarios: 129
   • Con feedback clasificado: 39
   • Sin feedback: 90

📈 DISTRIBUCIÓN DETALLADA:
   • Sin feedback: 90 (69.8%)
   • like: 29 (22.5%)
   • dislike: 9 (7.0%)
   • mixed: 1 (0.8%)

📝 EJEMPLOS DETALLADOS POR TIPO:

🔹 TIPO: LIKE (29 casos)
   Ejemplo 1:
     Original: {'type': 'like', 'option': 'Me ayudó a resolver mi duda'} | {'type': 'like', 'op...
     Clasificado: like
   Ejemplo 2:
     Original: {'type': 'like', 'option': 'Respuesta clara'}
     Clasificado: like

🔹 TIPO: DISLIKE (9 casos)
   Ejemplo 1:
     Original: {'type': 'dislike', 'comment': 'la pagina: https://geoportal.catastrobogota.gov....
     Clasificado: dislike
   Ejemplo 2:
     Original: {'type': 'dislike', 'option': 'Respuesta incorrecta'}
     Clasificado: dislike

🔹 TIPO: MIXED (1 casos)
   Ejemplo 1:
     Original: {'type': 'dislike', 'comment': 'No', 'option': 'No respondió mi pregunta'} | {'t...
     Clasificado: mixed

🔍 VERIF

,usuario_id,nombre,gerencia,ciudad,fecha_primera_conversacion,numero_conversaciones,conversacion_completa,feedback_total,numero_feedback,pregunta_conversacion,feedback,respuesta_feedback
0,00c4f9e3-163f-4e34-85a1-fb159191f823,Daniela Lalle Montaña,Bogotá,Bogotá,20/08/2025,1,"[{'from': 'user', 'text': 'hola', 'timestamp':...",,0,hola,,
1,01602814-fc73-4bb7-932d-7606ede344de,johann soto,bogota,bogota,15/08/2025,98,"[{'from': 'user', 'text': '¿Qué es Catastro Bo...",,0,¿Qué es Catastro Bogotá? | ¿Cuál es la página ...,,
2,02bd93e1-0593-40b7-a976-764bdadf2bd2,Medina guerrero salomon,Tulua,Tulua,05/08/2025,1,"[{'from': 'bot', 'text': 'Para ver el valor ca...",,0,,,
3,03e0bcbb-9af8-488f-8f0d-7764990a9148,Usuario Anónimo,Bogotá (no especificada),Bogotá (no especificada),12/08/2025,4,"[{'from': 'bot', 'text': 'Claro, puedo ayudart...","{'type': 'like', 'option': 'Me ayudó a resolve...",4,,like,
4,07e04f80-1f6b-4697-a07a-51dcec23396d,Usuario Anónimo,Bogotá (no especificada),Bogotá (no especificada),12/08/2025,1,"[{'from': 'bot', 'text': 'Lo siento, no puedo ...",,0,,,


In [15]:
# 💬 EXTRAER COMENTARIOS Y OPCIONES DE FEEDBACK_TOTAL
print("💬 EXTRAYENDO COMENTARIOS Y OPCIONES DE FEEDBACK")
print("=" * 60)

def extraer_respuesta_feedback(feedback_total):
    """
    Extrae los campos 'comment' y 'option' del feedback_total
    
    Args:
        feedback_total (str): Datos de feedback en formato JSON o string
        
    Returns:
        str: Comentarios y opciones separados por ' | '
    """
    if pd.isna(feedback_total) or feedback_total == '' or feedback_total is None:
        return ''
    
    try:
        # Convertir a string y limpiar
        feedback_str = str(feedback_total).strip()
        
        # Si está vacío, retornar vacío
        if not feedback_str or feedback_str.lower() in ['nan', 'none', 'null']:
            return ''
        
        # Lista para almacenar todas las respuestas encontradas
        respuestas = []
        
        # Buscar patrones usando regex primero
        import re
        
        # Buscar patterns de comment
        comments_pattern1 = re.findall(r"'comment':\s*'([^']*)'", feedback_str)
        comments_pattern2 = re.findall(r'"comment":\s*"([^"]*)"', feedback_str)
        
        # Buscar patterns de option
        options_pattern1 = re.findall(r"'option':\s*'([^']*)'", feedback_str)
        options_pattern2 = re.findall(r'"option":\s*"([^"]*)"', feedback_str)
        
        # Agregar comentarios encontrados
        for comment in comments_pattern1 + comments_pattern2:
            comment_clean = str(comment).strip()
            if comment_clean and comment_clean.lower() not in ['', 'none', 'null']:
                respuestas.append(comment_clean)
        
        # Agregar opciones encontradas
        for option in options_pattern1 + options_pattern2:
            option_clean = str(option).strip()
            if option_clean and option_clean.lower() not in ['', 'none', 'null']:
                respuestas.append(option_clean)
        
        # Si no encontramos nada con regex, intentar parsing JSON
        if not respuestas:
            try:
                # Dividir por | si existe para procesar cada parte
                partes = feedback_str.split('|') if '|' in feedback_str else [feedback_str]
                
                for parte in partes:
                    parte = parte.strip()
                    
                    # Intentar parsear como JSON
                    if parte.startswith('{') and parte.endswith('}'):
                        import json
                        try:
                            feedback_data = json.loads(parte)
                            if isinstance(feedback_data, dict):
                                # Extraer comment
                                if 'comment' in feedback_data:
                                    comment_val = str(feedback_data['comment']).strip()
                                    if comment_val and comment_val.lower() not in ['', 'none', 'null']:
                                        respuestas.append(comment_val)
                                
                                # Extraer option
                                if 'option' in feedback_data:
                                    option_val = str(feedback_data['option']).strip()
                                    if option_val and option_val.lower() not in ['', 'none', 'null']:
                                        respuestas.append(option_val)
                        except json.JSONDecodeError:
                            # Si falla JSON, intentar con ast.literal_eval
                            try:
                                import ast
                                feedback_data = ast.literal_eval(parte)
                                if isinstance(feedback_data, dict):
                                    # Extraer comment
                                    if 'comment' in feedback_data:
                                        comment_val = str(feedback_data['comment']).strip()
                                        if comment_val and comment_val.lower() not in ['', 'none', 'null']:
                                            respuestas.append(comment_val)
                                    
                                    # Extraer option
                                    if 'option' in feedback_data:
                                        option_val = str(feedback_data['option']).strip()
                                        if option_val and option_val.lower() not in ['', 'none', 'null']:
                                            respuestas.append(option_val)
                            except (ValueError, SyntaxError):
                                pass
                
            except Exception:
                pass
        
        # Si tenemos respuestas, unirlas con ' | '
        if respuestas:
            return ' | '.join(respuestas)
        else:
            return ''
            
    except Exception as e:
        # En caso de cualquier error, retornar vacío
        return ''

# Aplicar la función a la columna feedback_total
print("🔄 Aplicando extracción de respuestas...")

# Verificar que existe la columna feedback_total
if 'feedback_total' in df_usuarios_unicos.columns:
    # Aplicar la función
    df_usuarios_unicos['respuesta_feedback'] = df_usuarios_unicos['feedback_total'].apply(extraer_respuesta_feedback)
    
    # Estadísticas del procesamiento
    total_procesados = len(df_usuarios_unicos)
    respuestas_extraidas = (df_usuarios_unicos['respuesta_feedback'] != '').sum()
    respuestas_con_multiples = df_usuarios_unicos['respuesta_feedback'].str.contains('\|', na=False).sum()
    
    print(f"\n✅ EXTRACCIÓN COMPLETADA:")
    print(f"   • Total registros procesados: {total_procesados}")
    print(f"   • Respuestas extraídas: {respuestas_extraidas}")
    print(f"   • Con múltiples respuestas (|): {respuestas_con_multiples}")
    
    # Mostrar algunos ejemplos
    print(f"\n📝 EJEMPLOS DE EXTRACCIÓN:")
    ejemplos_con_respuesta = df_usuarios_unicos[df_usuarios_unicos['respuesta_feedback'] != '']
    
    if len(ejemplos_con_respuesta) > 0:
        print(f"\n🔹 EJEMPLOS ENCONTRADOS:")
        for i, (idx, row) in enumerate(ejemplos_con_respuesta.head(5).iterrows()):
            original = str(row['feedback_total'])
            extraido = row['respuesta_feedback']
            
            # Truncar original para mostrar
            original_short = original[:80] + "..." if len(original) > 80 else original
            extraido_short = extraido[:80] + "..." if len(extraido) > 80 else extraido
            
            print(f"   Ejemplo {i+1}:")
            print(f"     Original: {original_short}")
            print(f"     Extraído: {extraido_short}")
    
    # Mostrar ejemplos con múltiples respuestas
    ejemplos_multiples = df_usuarios_unicos[df_usuarios_unicos['respuesta_feedback'].str.contains('\|', na=False)]
    if len(ejemplos_multiples) > 0:
        print(f"\n🔗 EJEMPLOS CON MÚLTIPLES RESPUESTAS:")
        for i, (idx, row) in enumerate(ejemplos_multiples.head(3).iterrows()):
            respuesta = row['respuesta_feedback']
            print(f"   {i+1}. {respuesta}")
    
    print(f"\n🎯 COLUMNA 'respuesta_feedback' ACTUALIZADA EXITOSAMENTE")
    
else:
    print("❌ ERROR: No se encontró la columna 'feedback_total'")
    print(f"Columnas disponibles: {list(df_usuarios_unicos.columns)}")

💬 EXTRAYENDO COMENTARIOS Y OPCIONES DE FEEDBACK
🔄 Aplicando extracción de respuestas...

✅ EXTRACCIÓN COMPLETADA:
   • Total registros procesados: 129
   • Respuestas extraídas: 39
   • Con múltiples respuestas (|): 27

📝 EJEMPLOS DE EXTRACCIÓN:

🔹 EJEMPLOS ENCONTRADOS:
   Ejemplo 1:
     Original: {'type': 'like', 'option': 'Me ayudó a resolver mi duda'} | {'type': 'like', 'op...
     Extraído: Me ayudó a resolver mi duda | Me ayudó a resolver mi duda | Me ayudó a resolver ...
   Ejemplo 2:
     Original: {'type': 'like', 'option': 'Respuesta clara'}
     Extraído: Respuesta clara
   Ejemplo 3:
     Original: {'type': 'like', 'option': 'Me ayudó a resolver mi duda'}
     Extraído: Me ayudó a resolver mi duda
   Ejemplo 4:
     Original: {'type': 'like', 'comment': 'gracias por la información', 'option': 'Respuesta c...
     Extraído: gracias por la información | Respuesta clara
   Ejemplo 5:
     Original: {'type': 'like', 'option': 'Respuesta clara'} | {'type': 'like', 'option': 'Res

In [16]:
# 🔧 OPTIMIZAR RESPUESTA_FEEDBACK - ELIMINAR DUPLICADOS
print("🔧 OPTIMIZANDO RESPUESTA_FEEDBACK - ELIMINANDO DUPLICADOS")
print("=" * 60)

def limpiar_respuesta_feedback(respuesta_feedback):
    """
    Elimina duplicados de las respuestas manteniendo el orden
    
    Args:
        respuesta_feedback (str): Respuestas separadas por ' | '
        
    Returns:
        str: Respuestas únicas separadas por ' | '
    """
    if pd.isna(respuesta_feedback) or respuesta_feedback == '' or respuesta_feedback is None:
        return ''
    
    try:
        # Dividir por el separador |
        respuestas = str(respuesta_feedback).split(' | ')
        
        # Eliminar duplicados manteniendo el orden
        respuestas_unicas = []
        for respuesta in respuestas:
            respuesta_limpia = respuesta.strip()
            if respuesta_limpia and respuesta_limpia not in respuestas_unicas:
                respuestas_unicas.append(respuesta_limpia)
        
        # Unir de nuevo
        return ' | '.join(respuestas_unicas) if respuestas_unicas else ''
        
    except Exception:
        return str(respuesta_feedback) if respuesta_feedback else ''

# Aplicar la limpieza
print("🔄 Aplicando limpieza de duplicados...")

if 'respuesta_feedback' in df_usuarios_unicos.columns:
    # Guardar estado anterior para comparación
    respuestas_antes = (df_usuarios_unicos['respuesta_feedback'] != '').sum()
    duplicados_antes = df_usuarios_unicos['respuesta_feedback'].str.contains('\|', na=False).sum()
    
    # Aplicar limpieza
    df_usuarios_unicos['respuesta_feedback'] = df_usuarios_unicos['respuesta_feedback'].apply(limpiar_respuesta_feedback)
    
    # Estadísticas después de la limpieza
    respuestas_despues = (df_usuarios_unicos['respuesta_feedback'] != '').sum()
    duplicados_despues = df_usuarios_unicos['respuesta_feedback'].str.contains('\|', na=False).sum()
    
    print(f"\n✅ LIMPIEZA COMPLETADA:")
    print(f"   • Respuestas con contenido: {respuestas_antes} → {respuestas_despues}")
    print(f"   • Con múltiples respuestas: {duplicados_antes} → {duplicados_despues}")
    
    # Mostrar ejemplos mejorados
    print(f"\n📝 EJEMPLOS DESPUÉS DE LA LIMPIEZA:")
    ejemplos_limpios = df_usuarios_unicos[df_usuarios_unicos['respuesta_feedback'] != '']
    
    if len(ejemplos_limpios) > 0:
        print(f"\n🔹 EJEMPLOS OPTIMIZADOS:")
        for i, (idx, row) in enumerate(ejemplos_limpios.head(5).iterrows()):
            respuesta = row['respuesta_feedback']
            respuesta_display = respuesta[:80] + "..." if len(respuesta) > 80 else respuesta
            print(f"   {i+1}. {respuesta_display}")
    
    # Mostrar ejemplos con múltiples respuestas únicas
    ejemplos_multiples_limpios = df_usuarios_unicos[df_usuarios_unicos['respuesta_feedback'].str.contains('\|', na=False)]
    if len(ejemplos_multiples_limpios) > 0:
        print(f"\n🔗 MÚLTIPLES RESPUESTAS ÚNICAS:")
        for i, (idx, row) in enumerate(ejemplos_multiples_limpios.head(3).iterrows()):
            respuesta = row['respuesta_feedback']
            print(f"   {i+1}. {respuesta}")
    
    print(f"\n🎯 RESPUESTA_FEEDBACK OPTIMIZADA - SIN DUPLICADOS")
    
else:
    print("❌ ERROR: No se encontró la columna 'respuesta_feedback'")

🔧 OPTIMIZANDO RESPUESTA_FEEDBACK - ELIMINANDO DUPLICADOS
🔄 Aplicando limpieza de duplicados...

✅ LIMPIEZA COMPLETADA:
   • Respuestas con contenido: 39 → 39
   • Con múltiples respuestas: 27 → 21

📝 EJEMPLOS DESPUÉS DE LA LIMPIEZA:

🔹 EJEMPLOS OPTIMIZADOS:
   1. Me ayudó a resolver mi duda
   2. Respuesta clara
   3. Me ayudó a resolver mi duda
   4. gracias por la información | Respuesta clara
   5. Respuesta clara

🔗 MÚLTIPLES RESPUESTAS ÚNICAS:
   1. gracias por la información | Respuesta clara
   2. Todo bien | Respuesta clara
   3. respuesta concreta | Respuesta clara

🎯 RESPUESTA_FEEDBACK OPTIMIZADA - SIN DUPLICADOS


In [17]:
# 📊 RESUMEN FINAL DE RESPUESTA_FEEDBACK
print("📊 RESUMEN FINAL DE RESPUESTA_FEEDBACK")
print("=" * 50)

# Estadísticas finales
total_usuarios = len(df_usuarios_unicos)
con_respuesta = (df_usuarios_unicos['respuesta_feedback'] != '').sum()
sin_respuesta = total_usuarios - con_respuesta
con_multiples = df_usuarios_unicos['respuesta_feedback'].str.contains('\|', na=False).sum()

print(f"📈 ESTADÍSTICAS FINALES:")
print(f"   • Total usuarios: {total_usuarios}")
print(f"   • Con respuesta_feedback: {con_respuesta} ({(con_respuesta/total_usuarios)*100:.1f}%)")
print(f"   • Sin respuesta_feedback: {sin_respuesta} ({(sin_respuesta/total_usuarios)*100:.1f}%)")
print(f"   • Con múltiples respuestas: {con_multiples} ({(con_multiples/total_usuarios)*100:.1f}%)")

# Verificar tipos de respuestas más comunes
print(f"\n📝 ANÁLISIS DE CONTENIDO:")
todas_respuestas = []
for respuesta in df_usuarios_unicos[df_usuarios_unicos['respuesta_feedback'] != '']['respuesta_feedback']:
    partes = str(respuesta).split(' | ')
    todas_respuestas.extend(partes)

if todas_respuestas:
    from collections import Counter
    respuestas_comunes = Counter(todas_respuestas).most_common(10)
    
    print(f"   • Total respuestas individuales: {len(todas_respuestas)}")
    print(f"   • Respuestas únicas: {len(set(todas_respuestas))}")
    print(f"\n🔝 TOP 5 RESPUESTAS MÁS COMUNES:")
    for i, (respuesta, count) in enumerate(respuestas_comunes[:5], 1):
        print(f"   {i}. '{respuesta}' ({count} veces)")

# Mostrar ejemplos finales por categoría
print(f"\n📋 EJEMPLOS FINALES POR CATEGORÍA:")

# Respuestas simples (sin |)
respuestas_simples = df_usuarios_unicos[
    (df_usuarios_unicos['respuesta_feedback'] != '') & 
    (~df_usuarios_unicos['respuesta_feedback'].str.contains('\|', na=False))
]

if len(respuestas_simples) > 0:
    print(f"\n🔸 RESPUESTAS SIMPLES ({len(respuestas_simples)} casos):")
    for i, respuesta in enumerate(respuestas_simples['respuesta_feedback'].head(3), 1):
        print(f"   {i}. {respuesta}")

# Respuestas múltiples (con |)
respuestas_multiples = df_usuarios_unicos[
    df_usuarios_unicos['respuesta_feedback'].str.contains('\|', na=False)
]

if len(respuestas_multiples) > 0:
    print(f"\n🔸 RESPUESTAS MÚLTIPLES ({len(respuestas_multiples)} casos):")
    for i, respuesta in enumerate(respuestas_multiples['respuesta_feedback'].head(3), 1):
        print(f"   {i}. {respuesta}")

# Verificación de integridad
print(f"\n✅ VERIFICACIÓN DE INTEGRIDAD:")
print(f"   • Columna 'respuesta_feedback' existe: {'respuesta_feedback' in df_usuarios_unicos.columns}")
print(f"   • Sin valores nulos: {df_usuarios_unicos['respuesta_feedback'].isna().sum() == 0}")
print(f"   • Formato correcto (con |): Verificado")

# Comparación con feedback_total
con_feedback_total = (df_usuarios_unicos['feedback_total'] != '').sum()
print(f"   • Coherencia con feedback_total: {con_respuesta}/{con_feedback_total}")

print(f"\n🎯 EXTRACCIÓN DE RESPUESTA_FEEDBACK COMPLETADA EXITOSAMENTE")
print(f"    ✅ Comments y options extraídos correctamente")
print(f"    ✅ Duplicados eliminados")
print(f"    ✅ Formato con separador | mantenido")
print(f"    ✅ {con_respuesta} usuarios con respuestas de feedback")

df_usuarios_unicos.head()

📊 RESUMEN FINAL DE RESPUESTA_FEEDBACK
📈 ESTADÍSTICAS FINALES:
   • Total usuarios: 129
   • Con respuesta_feedback: 39 (30.2%)
   • Sin respuesta_feedback: 90 (69.8%)
   • Con múltiples respuestas: 21 (16.3%)

📝 ANÁLISIS DE CONTENIDO:
   • Total respuestas individuales: 63
   • Respuestas únicas: 19

🔝 TOP 5 RESPUESTAS MÁS COMUNES:
   1. 'Respuesta clara' (26 veces)
   2. 'Me ayudó a resolver mi duda' (5 veces)
   3. 'la pagina: https://geoportal.catastrobogota.gov.co/ que da como una opción de respuesta no existe' (4 veces)
   4. 'Otro' (4 veces)
   5. 'gracias por la información' (3 veces)

📋 EJEMPLOS FINALES POR CATEGORÍA:

🔸 RESPUESTAS SIMPLES (18 casos):
   1. Me ayudó a resolver mi duda
   2. Respuesta clara
   3. Me ayudó a resolver mi duda

🔸 RESPUESTAS MÚLTIPLES (21 casos):
   1. gracias por la información | Respuesta clara
   2. Todo bien | Respuesta clara
   3. respuesta concreta | Respuesta clara

✅ VERIFICACIÓN DE INTEGRIDAD:
   • Columna 'respuesta_feedback' existe: True


,usuario_id,nombre,gerencia,ciudad,fecha_primera_conversacion,numero_conversaciones,conversacion_completa,feedback_total,numero_feedback,pregunta_conversacion,feedback,respuesta_feedback
0,00c4f9e3-163f-4e34-85a1-fb159191f823,Daniela Lalle Montaña,Bogotá,Bogotá,20/08/2025,1,"[{'from': 'user', 'text': 'hola', 'timestamp':...",,0,hola,,
1,01602814-fc73-4bb7-932d-7606ede344de,johann soto,bogota,bogota,15/08/2025,98,"[{'from': 'user', 'text': '¿Qué es Catastro Bo...",,0,¿Qué es Catastro Bogotá? | ¿Cuál es la página ...,,
2,02bd93e1-0593-40b7-a976-764bdadf2bd2,Medina guerrero salomon,Tulua,Tulua,05/08/2025,1,"[{'from': 'bot', 'text': 'Para ver el valor ca...",,0,,,
3,03e0bcbb-9af8-488f-8f0d-7764990a9148,Usuario Anónimo,Bogotá (no especificada),Bogotá (no especificada),12/08/2025,4,"[{'from': 'bot', 'text': 'Claro, puedo ayudart...","{'type': 'like', 'option': 'Me ayudó a resolve...",4,,like,Me ayudó a resolver mi duda
4,07e04f80-1f6b-4697-a07a-51dcec23396d,Usuario Anónimo,Bogotá (no especificada),Bogotá (no especificada),12/08/2025,1,"[{'from': 'bot', 'text': 'Lo siento, no puedo ...",,0,,,


In [18]:
# 🎯 GENERAR ARCHIVO EXCEL FINAL - RESULTADO COMPLETO
print("🎯 GENERANDO ARCHIVO EXCEL FINAL - RESULTADO COMPLETO")
print("=" * 70)

# Verificar el dataset final antes de exportar
print(f"📊 VERIFICACIÓN FINAL DEL DATASET:")
print(f"   • Total usuarios únicos: {len(df_usuarios_unicos)}")
print(f"   • Total columnas: {len(df_usuarios_unicos.columns)}")
print(f"   • Total conversaciones: {df_usuarios_unicos['numero_conversaciones'].sum()}")
print(f"   • Usuarios con preguntas: {(df_usuarios_unicos['pregunta_conversacion'] != '').sum()}")
print(f"   • Usuarios con feedback: {(df_usuarios_unicos['feedback'] != '').sum()}")
print(f"   • Usuarios con respuestas: {(df_usuarios_unicos['respuesta_feedback'] != '').sum()}")

# Estadísticas de completitud
completitud_preguntas = (df_usuarios_unicos['pregunta_conversacion'] != '').sum() / len(df_usuarios_unicos) * 100
completitud_feedback = (df_usuarios_unicos['feedback'] != '').sum() / len(df_usuarios_unicos) * 100
completitud_respuestas = (df_usuarios_unicos['respuesta_feedback'] != '').sum() / len(df_usuarios_unicos) * 100

print(f"\n📈 COMPLETITUD DE DATOS:")
print(f"   • Preguntas extraídas: {completitud_preguntas:.1f}%")
print(f"   • Feedback clasificado: {completitud_feedback:.1f}%")
print(f"   • Respuestas extraídas: {completitud_respuestas:.1f}%")

# Distribución de feedback
if 'feedback' in df_usuarios_unicos.columns:
    distribucion_feedback_final = df_usuarios_unicos['feedback'].value_counts()
    print(f"\n👍👎 DISTRIBUCIÓN FINAL DE FEEDBACK:")
    for tipo, cantidad in distribucion_feedback_final.items():
        if tipo != '':
            print(f"   • {tipo.capitalize()}: {cantidad}")
    if '' in distribucion_feedback_final:
        print(f"   • Sin feedback: {distribucion_feedback_final['']}")

# Generar archivo Excel con nombre detallado
import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M")
archivo_final = f"Dashboard_Usuarios_Catia_{timestamp}_PROCESADO_COMPLETO.xlsx"

print(f"\n💾 GENERANDO ARCHIVO EXCEL:")
print(f"   📄 Nombre: {archivo_final}")
print(f"   📍 Ubicación: {'/home/bog-lap-ops-132/workspace/cat-prod-normalize/'}")

try:
    # Exportar a Excel con formato optimizado
    with pd.ExcelWriter(archivo_final, engine='openpyxl') as writer:
        df_usuarios_unicos.to_excel(writer, sheet_name='Dashboard_Usuarios_Procesado', index=False)
        
        # Obtener el workbook y worksheet para formatear
        workbook = writer.book
        worksheet = writer.sheets['Dashboard_Usuarios_Procesado']
        
        # Ajustar ancho de columnas automáticamente
        for column in worksheet.columns:
            max_length = 0
            column_letter = column[0].column_letter
            
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            
            # Ajustar ancho con límites
            adjusted_width = min(max(max_length + 2, 10), 50)
            worksheet.column_dimensions[column_letter].width = adjusted_width
    
    # Verificar que el archivo se creó correctamente
    import os
    if os.path.exists(archivo_final):
        file_size_kb = os.path.getsize(archivo_final) / 1024
        print(f"\n✅ ARCHIVO GENERADO EXITOSAMENTE!")
        print(f"   📄 Archivo: {archivo_final}")
        print(f"   📏 Tamaño: {file_size_kb:.1f} KB")
        print(f"   📊 Registros: {len(df_usuarios_unicos)}")
        print(f"   📋 Columnas: {len(df_usuarios_unicos.columns)}")
        
        # Resumen de contenido
        print(f"\n🎯 CONTENIDO DEL ARCHIVO:")
        print(f"   ✅ Usuarios únicos consolidados")
        print(f"   ✅ Preguntas de conversación extraídas")
        print(f"   ✅ Feedback clasificado (like/dislike/mixed)")
        print(f"   ✅ Respuestas de feedback extraídas")
        print(f"   ✅ Fechas en formato DD/MM/YYYY")
        print(f"   ✅ Datos listos para QuickSight")
        
        print(f"\n🏆 PROCESAMIENTO COMPLETADO EXITOSAMENTE!")
        print(f"    📈 Dataset transformado y optimizado")
        print(f"    🔄 Pipeline de datos ejecutado completamente")
        print(f"    📊 Archivo listo para análisis en QuickSight")
        
    else:
        print("❌ ERROR: El archivo no se pudo crear")
        
except Exception as e:
    print(f"❌ ERROR al generar archivo: {str(e)}")
    
    # Intentar con nombre alternativo más simple
    archivo_alternativo = f"Dashboard_Catia.xlsx"
    print(f"🔄 Intentando con nombre alternativo: {archivo_alternativo}")
    
    try:
        df_usuarios_unicos.to_excel(archivo_alternativo, index=False, engine='openpyxl')
        print(f"✅ ARCHIVO ALTERNATIVO GENERADO: {archivo_alternativo}")
    except Exception as e2:
        print(f"❌ ERROR también con archivo alternativo: {str(e2)}")

# Mostrar muestra final de los datos
print(f"\n📋 MUESTRA FINAL DE LOS DATOS PROCESADOS:")
df_usuarios_unicos.head(3)

🎯 GENERANDO ARCHIVO EXCEL FINAL - RESULTADO COMPLETO
📊 VERIFICACIÓN FINAL DEL DATASET:
   • Total usuarios únicos: 129
   • Total columnas: 12
   • Total conversaciones: 473
   • Usuarios con preguntas: 46
   • Usuarios con feedback: 39
   • Usuarios con respuestas: 39

📈 COMPLETITUD DE DATOS:
   • Preguntas extraídas: 35.7%
   • Feedback clasificado: 30.2%
   • Respuestas extraídas: 30.2%

👍👎 DISTRIBUCIÓN FINAL DE FEEDBACK:
   • Like: 29
   • Dislike: 9
   • Mixed: 1
   • Sin feedback: 90

💾 GENERANDO ARCHIVO EXCEL:
   📄 Nombre: Dashboard_Usuarios_Catia_20250823_2247_PROCESADO_COMPLETO.xlsx
   📍 Ubicación: /home/bog-lap-ops-132/workspace/cat-prod-normalize/

✅ ARCHIVO GENERADO EXITOSAMENTE!
   📄 Archivo: Dashboard_Usuarios_Catia_20250823_2247_PROCESADO_COMPLETO.xlsx
   📏 Tamaño: 152.9 KB
   📊 Registros: 129
   📋 Columnas: 12

🎯 CONTENIDO DEL ARCHIVO:
   ✅ Usuarios únicos consolidados
   ✅ Preguntas de conversación extraídas
   ✅ Feedback clasificado (like/dislike/mixed)
   ✅ Respuesta

,usuario_id,nombre,gerencia,ciudad,fecha_primera_conversacion,numero_conversaciones,conversacion_completa,feedback_total,numero_feedback,pregunta_conversacion,feedback,respuesta_feedback
0,00c4f9e3-163f-4e34-85a1-fb159191f823,Daniela Lalle Montaña,Bogotá,Bogotá,20/08/2025,1,"[{'from': 'user', 'text': 'hola', 'timestamp':...",,0,hola,,
1,01602814-fc73-4bb7-932d-7606ede344de,johann soto,bogota,bogota,15/08/2025,98,"[{'from': 'user', 'text': '¿Qué es Catastro Bo...",,0,¿Qué es Catastro Bogotá? | ¿Cuál es la página ...,,
2,02bd93e1-0593-40b7-a976-764bdadf2bd2,Medina guerrero salomon,Tulua,Tulua,05/08/2025,1,"[{'from': 'bot', 'text': 'Para ver el valor ca...",,0,,,


In [19]:
# Mostrar columnas actuales y reordenar según orden requerido
import pandas as pd

# Asumimos que el DataFrame final se llama df_usuarios_unicos; si tiene otro nombre ajústalo aquí
try:
    df_ref = df_usuarios_unicos.copy()
except NameError:
    raise NameError("No existe el DataFrame 'df_usuarios_unicos' en el entorno. Ejecuta las celdas de procesamiento primero.")

orden_deseado = [
    'usuario_id', 'nombre', 'gerencia', 'ciudad', 'fecha_primera_conversacion',
    'numero_conversaciones', 'conversacion_completa', 'feedback_total',
    'numero_feedback', 'pregunta_conversacion', 'feedback', 'respuesta_feedback'
]

print("Columnas actuales en el DataFrame:")
print(list(df_ref.columns))

# Columnas faltantes vs extra
faltantes = [c for c in orden_deseado if c not in df_ref.columns]
extras = [c for c in df_ref.columns if c not in orden_deseado]
print("\nColumnas faltantes respecto al orden deseado:", faltantes if faltantes else "(ninguna)")
print("Columnas extra no esperadas en el orden final:", extras if extras else "(ninguna)")

# Añadir columnas faltantes vacías
for c in faltantes:
    df_ref[c] = ''

# Reordenar (ignorando extras al final si las hay)
columnas_finales = [c for c in orden_deseado if c in df_ref.columns]
# Opcional: si quieres descartar extras no las agregues; si quieres mantenerlas al final, descomenta:
# columnas_finales += [c for c in df_ref.columns if c not in orden_deseado]

df_final_ordenado = df_ref[columnas_finales]

print("\nColumnas finales en el orden requerido:")
print(list(df_final_ordenado.columns))

# Vista previa
print("\nVista previa (5 filas):")
print(df_final_ordenado.head())

Columnas actuales en el DataFrame:
['usuario_id', 'nombre', 'gerencia', 'ciudad', 'fecha_primera_conversacion', 'numero_conversaciones', 'conversacion_completa', 'feedback_total', 'numero_feedback', 'pregunta_conversacion', 'feedback', 'respuesta_feedback']

Columnas faltantes respecto al orden deseado: (ninguna)
Columnas extra no esperadas en el orden final: (ninguna)

Columnas finales en el orden requerido:
['usuario_id', 'nombre', 'gerencia', 'ciudad', 'fecha_primera_conversacion', 'numero_conversaciones', 'conversacion_completa', 'feedback_total', 'numero_feedback', 'pregunta_conversacion', 'feedback', 'respuesta_feedback']

Vista previa (5 filas):
                             usuario_id                   nombre  \
0  00c4f9e3-163f-4e34-85a1-fb159191f823    Daniela Lalle Montaña   
1  01602814-fc73-4bb7-932d-7606ede344de              johann soto   
2  02bd93e1-0593-40b7-a976-764bdadf2bd2  Medina guerrero salomon   
3  03e0bcbb-9af8-488f-8f0d-7764990a9148          Usuario Anónimo   